In [1]:
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import torch
from torch import nn
from torch.utils.data import DataLoader
from datetime import datetime
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
from helper.label import classes
from helper.audio_extraction.get_file_list import get_file_list
from helper.audio_extraction.padded_and_windowed import extract_windowed_features
from FeedForward import ChordAI

In [3]:
TRAINING_DATA_PATH = './audio'
train_list = get_file_list(TRAINING_DATA_PATH)
data = extract_windowed_features(train_list, classes)

d:\pribadi\Chord Guitar AI\venv\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=690
  warnings.warn(
d:\pribadi\Chord Guitar AI\venv\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=345
  warnings.warn(


In [4]:
INPUT_SIZE = data[0][0].shape[0]
OUTPUT_SIZE = len(classes)
BATCH_SIZE = 32

In [5]:

model = ChordAI(INPUT_SIZE, OUTPUT_SIZE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [6]:
def create_data_loader(data, batch_size):
    dataloader = DataLoader(data, batch_size=batch_size)
    return dataloader

In [7]:
train_dataloader = create_data_loader(data, BATCH_SIZE)

In [8]:
def train_one_epoch(model, data_loader, loss_function, optimizer, device):
    acc = 0
    for inputs, targets in data_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        # change to tensor
        inputs = torch.tensor(inputs, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.long)
        
        # calculate loss
        predictions = model(inputs)
        loss = loss_function(predictions, targets)
        # backpropagate error and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # calculate accuracy
        acc += (predictions.argmax(1) == targets).sum().item()
    return acc / len(data_loader.dataset)

In [9]:
def train(model, train_data, loss_function, optimizer, device):
    tac = []
    i = 0
    patience = 0
    scr = 0
    start_time = datetime.now()
    while True:
        i+=1
        print(f"\nEpoch : {i:4} | ", end=" ")
        train_acc = train_one_epoch(model, train_data, loss_function, optimizer, device)
        tac.append(train_acc)

        print(f"train acc : {train_acc:.4f} | patience : {patience} | best acc : {scr:.4f}", end=" ") 
        if train_acc > scr:
            scr = train_acc
            patience = 0
            torch.save(model.state_dict(), "models/chord_model.pth")
            log = {
            "train_acc": tac,
            }
            torch.save(log, "models/logs.pth")
        else:
            patience +=1

        if patience >= 5:
            break
    end_time = datetime.now()
    print(f"\nTraining completed in {(end_time-start_time).seconds} seconds")
    torch.save({
        'INPUT_SIZE': INPUT_SIZE,
        'OUTPUT_SIZE': OUTPUT_SIZE,
    }, "models/config.pth")

In [10]:
train(model, train_dataloader, loss_fn, optimizer, device)


Epoch :    1 |  

C:\Users\arwan\AppData\Local\Temp\ipykernel_237760\3847291912.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
C:\Users\arwan\AppData\Local\Temp\ipykernel_237760\3847291912.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long)


train acc : 0.0145 | patience : 0 | best acc : 0.0000 
Epoch :    2 |  train acc : 0.0884 | patience : 0 | best acc : 0.0145 
Epoch :    3 |  train acc : 0.1684 | patience : 0 | best acc : 0.0884 
Epoch :    4 |  train acc : 0.3844 | patience : 0 | best acc : 0.1684 
Epoch :    5 |  train acc : 0.3912 | patience : 0 | best acc : 0.3844 
Epoch :    6 |  train acc : 0.5680 | patience : 0 | best acc : 0.3912 
Epoch :    7 |  train acc : 0.5723 | patience : 0 | best acc : 0.5680 
Epoch :    8 |  train acc : 0.6692 | patience : 0 | best acc : 0.5723 
Epoch :    9 |  train acc : 0.7976 | patience : 0 | best acc : 0.6692 
Epoch :   10 |  train acc : 0.8520 | patience : 0 | best acc : 0.7976 
Epoch :   11 |  train acc : 0.8673 | patience : 0 | best acc : 0.8520 
Epoch :   12 |  train acc : 0.9022 | patience : 0 | best acc : 0.8673 
Epoch :   13 |  train acc : 0.8971 | patience : 0 | best acc : 0.9022 
Epoch :   14 |  train acc : 0.8912 | patience : 1 | best acc : 0.9022 
Epoch :   15 |  train 